In [1]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [2]:
from neo4j import GraphDatabase

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

def get_ingredient_ratios(tx):
    query = """
    MATCH (p:ManufacturingProcess)-[r:USED_RATIO]->(i:Ingredient)
    RETURN i.name, r.ratio
    """
    result = tx.run(query)
    return [(record["i.name"], record["r.ratio"]) for record in result]

def cluster_products_by_ingredient_ratios(tx, min_ratio):
    query = """
    MATCH (p:ManufacturingProcess)-[r:USED_RATIO]->(i:Ingredient)
    WITH p, i.name as ingredient, r.ratio as ratio
    ORDER BY ratio DESC
    WITH collect({ingredient: ingredient, ratio: ratio}) as ingredient_ratios, p
    WITH [x in ingredient_ratios WHERE x.ratio >= $min_ratio] as common_ingredients, collect(p) as products
    RETURN common_ingredients, products
    """
    result = tx.run(query, min_ratio=min_ratio)
    return [(record["common_ingredients"], record["products"]) for record in result]

with driver.session() as session:
    print("Ingredient ratios:")
    for name, ratio in session.read_transaction(get_ingredient_ratios):
        print(f"{name}: {ratio}")

    print("\nProduct clusters by high ingredient ratios (min ratio = 0.3):")
    for ingredients, products in session.read_transaction(cluster_products_by_ingredient_ratios, 0.3):
        print(f"Common high-ratio ingredients: {[i['ingredient'] for i in ingredients]}")
        for product in products:
            print(f"  - {product['description']}")
        print()

driver.close()

GdsNotFound: The Graph Data Science library is not correctly installed on the Neo4j server.
                    Please refer to https://neo4j.com/docs/graph-data-science/current/installation/.
                    

In [ ]:
from neo4j import GraphDatabase


def calculate_research_ratios(tx):
    query = """
    MATCH (p:Product)-[:HAS_RECIPE]->()-[:USES_INGREDIENT]->()-[r:HAS_RESEARCH]->(res:IngredientResearch)
    WITH res, count(distinct p) as product_count, count(r) as total_count
    RETURN res.name AS research, toFloat(product_count) / toFloat(total_count) AS ratio
    ORDER BY ratio DESC
    """
    result = tx.run(query)
    return [(record["research"], record["ratio"]) for record in result]

def group_researches_by_frequency(tx):
    query = """
    MATCH (p:Product)-[:HAS_RECIPE]->()-[:USES_INGREDIENT]->()-[r:HAS_RESEARCH]->(res:IngredientResearch)
    WITH res, count(distinct p) as product_count, count(r) as total_count, toFloat(product_count) / toFloat(total_count) AS ratio
    RETURN
      CASE
        WHEN ratio > 0.8 THEN 'High Frequency'
        WHEN ratio > 0.5 THEN 'Medium Frequency'
        ELSE 'Low Frequency'
      END AS frequency_group,
      collect(res.name) AS researches,
      collect(ratio) AS ratios
    """
    result = tx.run(query)
    return [(record["frequency_group"], record["researches"], record["ratios"]) for record in result]

with driver.session() as session:
    print("IngredientResearch ratios:")
    for research, ratio in session.read_transaction(calculate_research_ratios):
        print(f"{research}: {ratio}")

    print("\nIngredientResearch groups by frequency:")
    for group, researches, ratios in session.read_transaction(group_researches_by_frequency):
        print(f"{group}:")
        for research, ratio in zip(researches, ratios):
            print(f"  {research}: {ratio}")

driver.close()

In [ ]:
query = """

MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount

MATCH (allProducts:Product)
WITH res, productCount, COUNT(allProducts) AS totalProductCount
RETURN res.ingredient_name AS IngredientResearch, toFloat(productCount) / toFloat(totalProductCount) AS FrequencyRatio

ORDER BY FrequencyRatio DESC

#All details the ratio more than 0.1

MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount
RETURN res.ingredient_name AS IngredientResearch, productCount

ORDER BY productCount DESC



MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, 0.1 AS threshold


WHERE frequencyRatio > threshold
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, ingredientCount, frequencyRatio

ORDER BY frequencyRatio DESC


#All ratios not only more than 0.1

MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount

MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, products, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, products, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, products, frequencyRatio
ORDER BY frequencyRatio DESC


"""

In [2]:
from neo4j import GraphDatabase
import plotly.graph_objects as go

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

# Define your Cypher query
query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, 0.1 AS threshold


WHERE frequencyRatio > threshold
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, ingredientCount, frequencyRatio
"""

# Execute the query
with driver.session() as session:
    result = session.run(query)
    data = result.data()

# Process the results for visualization
ingredient_names = [row['HighFrequencyIngredientResearch'] for row in data]
product_counts = [row['productCount'] for row in data]
ingredient_counts = [row['ingredientCount'] for row in data]
frequency_ratios = [row['frequencyRatio'] for row in data]

# Create a Plotly bar plot
fig = go.Figure(data=[go.Bar(x=ingredient_names, y=frequency_ratios)])
fig.update_layout(title='High Frequency Ingredient Research', xaxis_title='Ingredient Research', yaxis_title='Frequency Ratio')

# Show the plot
fig.show()


Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('9d1381c2.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))


In [3]:
from neo4j import GraphDatabase
import pandas as pd



# Define your Cypher query
query = """
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, p
ORDER BY p.name, res.ingredient_name

RETURN res.ingredient_name AS HighFrequencyIngredientResearch, COLLECT(p.name) AS Products
"""

# Execute the query
with driver.session() as session:
    result = session.run(query)
    data = result.data()

# Create a DataFrame from the results
df = pd.DataFrame(data)

# Group products by HighFrequencyIngredientResearch
grouped_products = df.groupby('HighFrequencyIngredientResearch')['Products'].apply(list).reset_index()

# Print the grouped products
print(grouped_products)


    HighFrequencyIngredientResearch  \
0                                     
1              Acesulfame Potassium   
2                       Acetic Acid   
3             Acidity Regulator 331   
4             Acidity Regulator 339   
..                              ...   
303                Whole Egg Powder   
304                     Xanthan Gum   
305                           Yeast   
306                   Yeast Extract   
307                            Zinc   

                                              Products  
0    [[Rimmel Scandaleyes Eye Definer Brown, Rimmel...  
1    [[Coca - Cola Caffeine Free Zero Sugar Soft Dr...  
2    [[Fountain Barbecue Bbq Sauce Squeeze, Fountai...  
3     [[Cadbury Dairy Milk Caramello Koala Chocolate]]  
4                          [[Woolworths Creamed Rice]]  
..                                                 ...  
303              [[Woolworths Chocolate Chip Cookies]]  
304  [[Fountain Barbecue Bbq Sauce Squeeze, Woolwor...  
305  [[Kitkat 4 Fi

In [16]:
import plotly.graph_objects as go
import pandas as pd
from neo4j import GraphDatabase


# Define your Cypher query
query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, 0.1 AS threshold

WHERE frequencyRatio > threshold
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, ingredientCount, frequencyRatio
"""

# Execute the query
with driver.session() as session:
    result = session.run(query)
    data = result.data()

# Create a DataFrame from the results
df = pd.DataFrame(data)

# Group products by HighFrequencyIngredientResearch
grouped_products = df.groupby('HighFrequencyIngredientResearch')['productCount'].apply(list).reset_index()

# Create a Plotly scatter plot
fig = go.Figure()

for index, row in grouped_products.iterrows():
    x = row['productCount']
    y = [row['HighFrequencyIngredientResearch']] * len(x)
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name=row['HighFrequencyIngredientResearch']))

fig.update_layout(title='Products Grouped by High Frequency Ingredient Research', xaxis_title='Products Count', yaxis_title='High Frequency Ingredient Research')

# Show the plot
fig.show()



In [29]:
import plotly.graph_objects as go
from neo4j import GraphDatabase




query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COUNT(DISTINCT i) AS ingredientCount, COLLECT(p.name) AS Products

MATCH (allProducts:Product)
WITH res, productCount, ingredientCount, COUNT(allProducts) AS totalProductCount, Products
WITH res, productCount, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, 0.1 AS threshold, Products

WHERE frequencyRatio > threshold
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, ingredientCount, frequencyRatio, Products
"""


with driver.session() as session:
    result = session.run(query)
    data = result.data()


fig = go.Figure()

for record in data:
    ingredient_research = record['HighFrequencyIngredientResearch']
    products = record['Products']
    for product in products:
        fig.add_trace(go.Scatter(x=[product], y=[ingredient_research], mode='markers', name=ingredient_research))

fig.update_layout(title='Products Grouped by High Frequency Ingredient Research', xaxis_title='Products', yaxis_title='High Frequency Ingredient Research')


fig.show()


In [41]:
from neo4j import GraphDatabase




query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, 0.1 AS threshold
WHERE frequencyRatio > threshold
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res)
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, collect(p.name) AS Products

"""


with driver.session() as session:
    result = session.run(query)
    data = result.data()

grouped_products = {}


for record in data:
    ingredient_research = record['HighFrequencyIngredientResearch']
    products = record['Products']
    grouped_products[ingredient_research] = products


for ingredient_research, products in grouped_products.items():
    print(f"HighFrequencyIngredientResearch：{ingredient_research}")
    print("Product list：")
    for product in products:
        print(product)
    print()


HighFrequencyIngredientResearch：Sugar
Product list：
Coca - Cola Classic  Soft Drink Multipack Cans
Coca - Cola Classic  Soft Drink Multipack Cans
Coca - Cola Classic  Soft Drink Multipack Cans
Coca - Cola Classic  Soft Drink Mini Cans
Coca - Cola Vanilla  Soft Drink Multipack Cans
Indomie Mi Goreng Instant Noodles
Maggi 2 Minute Chicken Flavour Instant Noodles
Maggi 2 Minute Chicken Flavour Instant Noodles
Supreme Noodles Cup Chicken Flavour
Fantastic Chicken Noodle Cup
Fantastic Chicken Noodle Bowl
Indomie Mi Goreng Noodle Cup
Maggi 2 Minute Beef Flavour Instant Noodles
Supreme Noodles Cup Beef Flavour
Schweppes Indian Tonic Water Classic Mixers Bottle
Schweppes Lime Soda Water With Lime Juice Bottle
Schweppes Lemon Lime Natural Mineral Water Bottle
Schweppes Orange Mango Natural Mineral Water Bottle
Kitkat 4 Finger Milk Chocolate Bar
Kitkat Chunky Original Chocolate Bar
Cadbury Dairy Milk Caramello Koala Chocolate
Mars Chocolate Bar With Nougat & Caramel
Cadbury Dairy Milk Chocolate 

In [85]:
import plotly.graph_objects as go
from neo4j import GraphDatabase



query = """
MATCH (p:Product)
WITH count(p) AS totalProductCount
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, totalProductCount, 0.1 AS threshold
WITH res, productCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, threshold
WHERE frequencyRatio > threshold
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res)
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, collect(p.name) AS Products

"""


with driver.session() as session:
    result = session.run(query)
    data = result.data()


high_frequency_ingredient_research = []
product_lists = []

for record in data:
    high_frequency_ingredient_research.append(record['HighFrequencyIngredientResearch'])
    product_lists.append(record['Products'])


fig = go.Figure()

for ingredient_research, products in zip(high_frequency_ingredient_research, product_lists):
    fig.add_trace(go.Table(
        header=dict(values=['HighFrequencyIngredientResearch', 'Products'],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[[ingredient_research], [", ".join(products)]],
                   fill_color='lavender',
                   align='left'))
    )

fig.update_layout(title='High Frequency Ingredient Research and Associated Products')


fig.show()


In [86]:
import plotly.graph_objects as go
import pandas as pd
from neo4j import GraphDatabase

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

df = pd.DataFrame(neo4j_data)


query = """
MATCH (p:Product)
WITH count(p) AS totalProductCount
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, totalProductCount, 0.1 AS threshold
WITH res, productCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio, threshold
WHERE frequencyRatio > threshold
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res)
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, collect(p.name) AS Products
"""



df['GroupNumber'] = df.groupby('HighFrequencyIngredientResearch').ngroup() + 1
df['GroupNumber'] = df['GroupNumber'].max() - df['GroupNumber'] + 1
df['GroupNumber'] = 'Group ' + df['GroupNumber'].astype(str)


df.sort_values(by='HighFrequencyIngredientResearch', ascending=False, inplace=True)

fig_data = [go.Table(
    header=dict(values=['HighFrequencyIngredientResearch', 'Products', 'GroupNumber']),
    cells=dict(values=[df['HighFrequencyIngredientResearch'], df['Products'], df['GroupNumber']])
)]

fig_layout = go.Layout(
    title='High Frequency Ingredient Research and Associated Products by Group (Descending Order)'
)

fig = go.Figure(data=fig_data, layout=fig_layout)
fig.show()


In [92]:
import plotly.graph_objects as go
import pandas as pd
from neo4j import GraphDatabase

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

df = pd.DataFrame(neo4j_data)


query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, products, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, products, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, products, ingredientCount, frequencyRatio
ORDER BY frequencyRatio DESC

"""

import plotly.graph_objects as go
import pandas as pd



df = pd.DataFrame(data)


df['GroupNumber'] = df.groupby('HighFrequencyIngredientResearch').ngroup() + 1
df['GroupNumber'] = df['GroupNumber'].max() - df['GroupNumber'] + 1
df['GroupNumber'] = 'Group ' + df['GroupNumber'].astype(str)
df.sort_values(by='HighFrequencyIngredientResearch', ascending=False, inplace=True)


fig_data = [go.Table(
    header=dict(values=['HighFrequencyIngredientResearch', 'Products', 'GroupNumber']),
    cells=dict(values=[df['HighFrequencyIngredientResearch'], df['Products'], df['GroupNumber']])
)]


fig_layout = go.Layout(
    title='High Frequency Ingredient Research and Associated Products by Group (Descending Order)'
)


fig = go.Figure(data=fig_data, layout=fig_layout)
fig.show()



In [95]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.graph_objects as go


query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, products, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, products, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, products, frequencyRatio
ORDER BY frequencyRatio DESC
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)


df['GroupNumber'] = df.groupby('HighFrequencyIngredientResearch').ngroup() + 1
df['GroupNumber'] = df['GroupNumber'].max() - df['GroupNumber'] + 1
df['GroupNumber'] = 'Group ' + df['GroupNumber'].astype(str)
df.sort_values(by='HighFrequencyIngredientResearch', ascending=False, inplace=True)


fig_data = go.Table(
    header=dict(values=['HighFrequencyIngredientResearch', 'Product Count', 'Products', 'Frequency Ratio', 'GroupNumber']),
    cells=dict(values=[df['HighFrequencyIngredientResearch'], df['productCount'], df['products'], df['frequencyRatio'],df['GroupNumber']])
)


fig_layout = go.Layout(
    title='High Frequency Ingredient Research and Associated Products',
)


fig = go.Figure(data=fig_data, layout=fig_layout)
fig.show()


In [98]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.graph_objects as go


query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, products, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, products, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, products, frequencyRatio
ORDER BY frequencyRatio DESC
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)


df.sort_values(by='frequencyRatio', ascending=False, inplace=True)


df['group'] = 'Group ' + (df['frequencyRatio'].rank(ascending=False, method='dense').astype(int)).astype(str)


fig_data = go.Table(
    header=dict(values=['HighFrequencyIngredientResearch', 'Product Count', 'Products', 'Frequency Ratio', 'Group']),
    cells=dict(values=[df['HighFrequencyIngredientResearch'], df['productCount'], df['products'], df['frequencyRatio'], df['group']])
)


fig_layout = go.Layout(
    title='High Frequency Ingredient Research and Associated Products (Sorted by Frequency Ratio)',
)


fig = go.Figure(data=fig_data, layout=fig_layout)
fig.show()


In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.graph_objects as go


query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, products, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, products, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, products, frequencyRatio
ORDER BY frequencyRatio DESC
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)


fig_data = go.Table(
    header=dict(values=['HighFrequencyIngredientResearch', 'Product Count', 'Products', 'Frequency Ratio']),
    cells=dict(values=[df['HighFrequencyIngredientResearch'], df['productCount'], df['products'], df['frequencyRatio']])
)


fig_layout = go.Layout(
    title='High Frequency Ingredient Research and Associated Products',
)


fig = go.Figure(data=fig_data, layout=fig_layout)
fig.show()


In [9]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.graph_objects as go


query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COLLECT(DISTINCT p.name) AS products, COUNT(DISTINCT i) AS ingredientCount
MATCH (allProducts:Product)
WITH res, productCount, products, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, products, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, productCount, products, frequencyRatio
ORDER BY frequencyRatio DESC
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]
    df = pd.DataFrame(data)

# The results are sorted according to the value of the frequency ratio (ratio) from highest to lowest.
df.sort_values(by='frequencyRatio', ascending=False, inplace=True)

# Assign a group name to each group（group）
df['group'] = 'Group ' + (df['HighFrequencyIngredientResearch'].rank(ascending=True, method='dense').astype(int)).astype(str)


fig_data = go.Table(
    header=dict(values=['HighFrequencyIngredientResearch', 'Product Count', 'Products', 'Frequency Ratio', 'Group']),
    cells=dict(values=[df['HighFrequencyIngredientResearch'], df['productCount'], df['products'], df['frequencyRatio'], df['group']])
)


fig_layout = go.Layout(
    title='High Frequency Ingredient Research and Associated Products (Sorted by Frequency Ratio)',
)


fig = go.Figure(data=fig_data, layout=fig_layout)
fig.show()

# Storing the results in a new DataFrame
data_lists = fig_data.cells.values
new_df = pd.DataFrame({
    'HighFrequencyIngredientResearch': data_lists[0],
    'Product Count': data_lists[1],
    'Products': data_lists[2],
    'Frequency Ratio': data_lists[3],
    'Group': data_lists[4]
})

print(new_df)

           HighFrequencyIngredientResearch  Product Count  \
0                                    Sugar             66   
1                                     Salt             50   
2                            Vegetable Oil             29   
3                              Wheat Flour             27   
4                                  Flavour             25   
..                                     ...            ...   
334                             Strawberry              1   
335                         Stabiliser 422              1   
336                            Citric Acid              1   
337                         Thickener 1414              1   
338  Clarified Pineapple Juice Concentrate              1   

                                              Products  Frequency Ratio  \
0    [Coca - Cola Classic  Soft Drink Multipack Can...         0.318841   
1    [Indomie Mi Goreng Instant Noodles, Maggi 2 Mi...         0.241546   
2    [Maggi 2 Minute Chicken Flavour Insta

In [12]:
with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

df = pd.DataFrame(data)

# Create groups from dfi
def add_group(tx, group_name):
    tx.run("CREATE (:Group {name: $name})", name=Group_name)

# Connect to Neo4j and create groupss
with GraphDatabase.driver() as driver:
    with driver.session() as session:
        for ingredient_name in df['U-Groups'].unique():  # Ensure unique group names
            session.execute_write(add_group, group_name)


            

TypeError: GraphDatabase.driver() missing 1 required positional argument: 'uri'

In [15]:
#create relationship between products and ingredients called "made of"
def create_relationship(tx, product_name, group_name):
    query = (
        "MATCH (p:Product {name: $product_name}), (gi:Group {name: $group_name}) "
        "MERGE (p)-[:GROUP_OF]->(gi)"
    )
    tx.run(query, product_name=product_name, group_name=group_name)

with driver.session() as session:
    # Iterate over each row in the DataFrame
    for index, row in new_df.iterrows():
        product_name = row["Name"]  
        for i in range(1, 151):  
            group_col = f"Group - {i}"
            if pd.notna(row[group_col]):  # Check if the ingredient cell is not empty
                session.write_transaction(create_relationship, product_name, row[group_col])



KeyError: 'product_name'

In [42]:
import plotly.graph_objects as go
from neo4j import GraphDatabase



query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch)
WITH res, COUNT(DISTINCT p) AS productCount, COUNT(DISTINCT i) AS ingredientCount

MATCH (allProducts:Product)
WITH res, productCount, ingredientCount, COUNT(allProducts) AS totalProductCount
WITH res, productCount, ingredientCount, totalProductCount, toFloat(productCount) / toFloat(totalProductCount) AS frequencyRatio
WHERE frequencyRatio > 0.1
MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:USES_INGREDIENT]->(:Ingredient)-[:HAS_RESEARCH]->(res)
RETURN res.ingredient_name AS HighFrequencyIngredientResearch, collect(p.name) AS Products, frequencyRatio

"""


with driver.session() as session:
    result = session.run(query)
    data = result.data()

ingredient_researches = [record['HighFrequencyIngredientResearch'] for record in data]
product_counts = [record['frequencyRatio'] for record in data]


fig = go.Figure(data=go.Scatter(x=product_counts, y=ingredient_researches, mode='markers', marker=dict(color='skyblue')))
fig.update_layout(title='High Frequency Ingredient Research', xaxis_title='Frequency Ratio', yaxis_title='Ingredient Research')
fig.update_yaxes(categoryorder='total ascending')


fig.show()


In [17]:
from neo4j import GraphDatabase


# Define functions to create group nodes
def create_group(tx, group_name):
    tx.run("CREATE (:Group {name: $group_name})", group_name=group_name)

# Define functions to create relationships
def create_relationship(tx, product_name, group_name):
    tx.run("""
    MATCH (p:Product {name: $product_name})
    MATCH (g:Group {name: $group_name})
    CREATE (p)-[:GROUP_TO]->(g)
    """, product_name=product_name, group_name=group_name)

# Extracting grouping information from a DataFrame and saving it to a Neo4j database
with driver.session() as session:
    for group_name, products in new_df.groupby('Group'):
        # Create group nodes
        session.write_transaction(create_group, group_name)
        
        # Create relationships for each product
        for product in products['Products']:
            session.write_transaction(create_relationship, product, group_name)


C:\Users\apple\AppData\Local\Temp\ipykernel_11588\319053344.py:20: DeprecationWarning:

write_transaction has been renamed to execute_write

C:\Users\apple\AppData\Local\Temp\ipykernel_11588\319053344.py:24: DeprecationWarning:

write_transaction has been renamed to execute_write

